In [1]:
POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres'
POSTGRES_PASSWORD = 'INSERT_PASSWORD_HERE'
POSTGRES_DBNAME = 'postgres'

api_key = 'INSERT_PUBLIC_KEY_HERE'

In [2]:
bitfinex_pairs = ['bateth', 'batusd', 'bchbtc', 'bchusd', 'bchusdt', 'btcusd',
                  'btcusdt', 'dashbtc', 'dashusd', 'eosbtc', 'eosusd',
                  'eosusdt', 'etcbtc', 'etcusd', 'ethbtc', 'ethusd', 'gntusd',
                  'ltcbtc', 'ltcusd', 'ltcusdt', 'manausd', 'repbtc',
                  'repusd', 'xlmbtc', 'xlmusd', 'xrpbtc', 'xrpusd', 'zecbtc',
                  'zecusd', 'zrxbtc', 'zrxusd']

coinbase_pro_pairs = ['bateth', 'batusdc', 'bchbtc', 'bchusd', 'btcusd', 
                      'btcusdc', 'dashbtc', 'dashusd', 'eosbtc', 'eosusd',
                      'etcbtc', 'etcusd', 'ethbtc', 'ethusd', 'ethusdc',
                      'gntusdc', 'ltcbtc', 'ltcusd', 'manausdc', 'repbtc',
                      'repusd', 'xlmbtc', 'xlmusd', 'xrpbtc', 'xrpusd',
                      'zecbtc', 'zecusdc', 'zrxbtc', 'zrxusd']

poloniex_pairs = ['bateth', 'batusdt', 'bchbtc', 'bchusdc', 'bchusdt', 
                  'btcusdc', 'btcusdt', 'dashbtc', 'dashusdc', 'dashusdt',
                  'eosbtc', 'eosusdc', 'eosusdt', 'etcbtc', 'etcusdc', 
                  'etcusdt', 'ethbtc', 'ethusdc', 'ethusdt', 'gntusdt', 
                  'ltcbtc', 'ltcusdc', 'ltcusdt', 'manausdt', 'repbtc',
                  'repusdt', 'xlmbtc', 'xlmusdc', 'xlmusdt', 'xrpbtc',
                  'xrpusdc', 'xrpusdt', 'zecbtc', 'zecusdc', 'zecusdt', 
                  'zrxbtc', 'zrxusdt']

exchanges = {'bitfinex': bitfinex_pairs,
             'coinbase-pro': coinbase_pro_pairs,
             'poloniex': poloniex_pairs}

tables = {}

for exchange in exchanges:
    tables[exchange] = {pair: ('_'.join(exchange.split('-')) + '_' + pair)
                        for pair in exchanges[exchange]}

In [3]:
import time
import requests
import psycopg2 as ps

base_url = ('https://api.cryptowat.ch/markets/{exchange}/{trading_pair}/' +
            'ohlc?apikey={api_key}&periods={periods}&after={after}')
periods = '60'
after = round(time.time()) - 200
    
def insert_data(tables, api_key, periods, after):
    
    conn = ps.connect(host=POSTGRES_ADDRESS,
                      database=POSTGRES_DBNAME,
                      user=POSTGRES_USERNAME,
                      password=POSTGRES_PASSWORD,
                      port=POSTGRES_PORT)
    
    cur = conn.cursor()

    for exchange in tables:
        for trading_pair in tables[exchange]:
            table_name = tables[exchange][trading_pair]
            
            retrieval_time = time.time()
            url = base_url.format(exchange=exchange,
                                  trading_pair=trading_pair,
                                  api_key=api_key,
                                  periods=periods,
                                  after=after)
            
            try:
                response = requests.get(url).json()
                candles = response['result'][periods]
                
                cur.execute("SELECT time FROM {table_name}".format(
                    table_name=table_name))
                results = cur.fetchall()
                timestamps = [result[0] for result in results]
                
                second_to_last_candle = candles[-2]
                if second_to_last_candle[0] not in timestamps:
                    new_data = second_to_last_candle[:6]
                    insert_query = """INSERT INTO {table_name}
                                   (time, open, high, low, close, volume)
                                   VALUES (%s, %s, %s, %s, %s, %s)
                                   """.format(table_name=table_name)
                    cur.execute(insert_query, new_data)
                    
                cur.execute("SELECT time FROM {table_name}".format(
                    table_name=table_name))
                results = cur.fetchall()
                timestamps = [result[0] for result in results]
                
                last_candle = candles[-1]
                if last_candle[0] + 60 < retrieval_time():
                    if last_candle[0] not in timestamps:
                        new_data = last_candle[:6]
                        print(new_data)
                        insert_query = """INSERT INTO {table_name}
                                       (time, open, high, low, close, volume)
                                       VALUES (%s, %s, %s, %s, %s, %s)
                                       """.format(table_name=table_name)
                        cur.execute(insert_query, new_data)
                        
            except:
                pass
            
    conn.commit()
    cur.close()
                
start = time.time()
while time.time() < start + 86400:
    insert_data(tables=tables, api_key=api_key, periods=periods, after=after)
    time.sleep(2)